# Loading CSV data in SecretFlow

The following codes are demos only. It's **NOT for production** due to system security concerns, please **DO NOT** use it directly in production.

This tutorial will demonstrate, through several examples, how to load CSV data in SecretFlow and utilize it for data processing and modeling.

## Setting up the environment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()
sf.init(['alice', 'bob', 'charlie'], address="local", log_to_driver=True)
alice, bob, charlie = sf.PYU('alice'), sf.PYU('bob'), sf.PYU('charlie')

The version of SecretFlow: 1.7.0.dev$$DATE$$


/home/beng003/anaconda/envs/sf/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
/home/beng003/anaconda/envs/sf/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-07-13 17:11:51,934	INFO worker.py:1724 -- Started a local Ray instance.


## Introduction to the interface

In SecretFlow, we provide an interface similar to `pandas.read_csv` to read CSV data from different parties and unify it into a federated concept of data.

- For horizontal scenarios, there is `secretflow.horizontal.read_csv` [API](https://www.secretflow.org.cn/docs/secretflow/en/source/secretflow.data.horizontal.html#secretflow.data.horizontal.read_csv) available.
- For horizontal scenarios, there is `secretflow.vertical.read_csv` [API](https://www.secretflow.org.cn/docs/secretflow/en/source/secretflow.data.horizontal.html#secretflow.data.vertical.read_csv) available. 

By using `read_csv`, you can read CSV files from multiple parties and create a FedDataFrame.

**Build Federated Table**  
A federated table is a virtual concept that spans multiple parties.

1. Data from each party in the federated table is stored locally and is not allowed to leave the domain.
2. Except for the party that owns the data, no one else can access the data storage.
3. Any operation on the federated table is scheduled by the Driver to each Worker, and the execution instructions are passed layer by layer until they reach the Python Runtime of the specific Worker that owns the data. The framework ensures that data can only be operated on when the Worker.device and Object.device are the same.
4. The federated table is designed to manage and manipulate multi-party data from a central perspective.
5. The interface aligns with `pandas.DataFrame` to reduce the cost of multi-party data operations.

<img alt="vdataframe.png" src="resources/vdataframe.png" width="600">  

## Data Download and Splitting

In [3]:
%%capture
%%!
wget https://secretflow-data.oss-accelerate.aliyuncs.com/datasets/iris/iris.csv

In [4]:
import pandas as pd

alldata_df = pd.read_csv("./iris.csv")

In [5]:
len(alldata_df)

150

In [6]:
alldata_df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [7]:
h_alice_df = alldata_df.loc[:70]
h_bob_df = alldata_df.loc[70:]

Save horizontally split DataFrame as CSV files separately.

In [8]:
# save the data to local file system
import tempfile

t1, h_alice_path = tempfile.mkstemp()
t2, h_bob_path = tempfile.mkstemp()
h_alice_df.to_csv(h_alice_path, index=False)
h_bob_df.to_csv(h_bob_path, index=False)

In [9]:
t1,t2

(80, 81)

In [10]:
v_alice_df = alldata_df.loc[:, ['sepal_length', 'sepal_width']]
v_bob_df = alldata_df.loc[:, ['petal_length', 'petal_width', 'class']]

Save vertically split DataFrame as CSV files separately.

In [11]:
# save the data to local file system
_, v_alice_path = tempfile.mkstemp()
_, v_bob_path = tempfile.mkstemp()
v_alice_df.to_csv(v_alice_path, index=True, index_label="id")
v_bob_df.to_csv(v_bob_path, index=True, index_label="id")

## Loading CSV Data Example for Horizontal Scenario

In [12]:
from secretflow.data.horizontal import read_csv
from secretflow.security.aggregation.plain_aggregator import PlainAggregator
from secretflow.security.compare.plain_comparator import PlainComparator
from secretflow.data.split import train_test_split

First, prepare the CSV data files for two parties. In a horizontal scenario, it is required that the schema of the data from both parties is consistent.

- Alice: datapath (the local path accessible on Alice's machine)
- Bob: datapath (the local path accessible on Bob's machine)

In a horizontal scenario, since the data with the same schema is distributed across multiple parties, cross-domain computation is required for certain DataFrame operations.
The `read_csv` interface requires the specification of an `aggregator` and a `comparator`. We can specify a `secure aggregator` and a `secure comparator` during computation to protect data privacy.

In [13]:
path_dict = {alice: h_alice_path, bob: h_bob_path}

aggregator = PlainAggregator(charlie)
comparator = PlainComparator(charlie)

hdf = read_csv(filepath=path_dict, aggregator=aggregator, comparator=comparator)

INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party bob.


In [14]:
hdf.columns

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

The resulting hdf is a FedDataFrame, and we can now perform some data processing on it.
For reference, [Data Preprocessing with FedDataFrame](https://www.secretflow.org.cn/docs/secretflow/en/tutorial/data_preprocessing_with_data_frame.html)：

In [15]:
label = hdf["class"]
data = hdf.drop(columns="class")

The obtained `data` and `label` can be inputted into FLModel or SLModel for modeling purposes.

SecretFlow provides `train_test_split` to split data.

In [19]:
train_data, test_data = train_test_split(
    data, train_size=0.8, shuffle=True, random_state=1234
)

In [20]:
print(train_data.partition_shape(), test_data.partition_shape())

{PYURuntime(alice): (56, 4), PYURuntime(bob): (64, 4)} {PYURuntime(alice): (15, 4), PYURuntime(bob): (16, 4)}


## Loading CSV Data Example for Vertical Scenario

First, prepare the CSV data files for two parties. In a vertical scenario, it is not required that the schema of the data from both parties is consistent.
We provide the ability of PSI in the `read_csv` interface.

- Alice: datapath (The local path accessible on Alice's machine)
- Bob: datapath (The local path accessible on Bob's machine)

In a vertical scenario, the schema of the data from both parties is not consistent, but each party has all the data of each column. No comparator or aggregator is required. However, the data of each party is not necessarily aligned, and we need to align the data through `PSI` when reading.


- path_dict: data path
- spu: spu device used for intersection
- keys: keys for intersection (support multi-column intersection)
- drop_keys: ID column name to be deleted after intersection

In [21]:
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))

In [22]:
spu

In [23]:
from secretflow.data.vertical import read_csv

In [25]:
path_dict = {
    alice: v_alice_path,  # The path that alice can access
    bob: v_bob_path,  # The path that bob can access
}

# Prepare the SPU device
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))

vdf = read_csv(path_dict, spu=spu, keys='id', drop_keys="id")

INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party bob.


In [30]:
vdf

VDataFrame(partitions={PYURuntime(alice): <secretflow.data.core.partition.Partition object at 0x7f7ebc4833d0>, PYURuntime(bob): <secretflow.data.core.partition.Partition object at 0x7f7ebc480dc0>}, aligned=True)

In [26]:
vdf.columns

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [27]:
label = vdf["class"]
data = vdf.drop(columns="class")

We can also use `train_test_split` to split data.

In [28]:
train_data, test_data = train_test_split(
    data, train_size=0.8, shuffle=True, random_state=1234
)

## Next, you can try your own csv data

In [29]:
sf.shutdown()